In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
os.chdir("..")

DATA_DIR = os.getenv("DATA_DIR")
OUTPUT_DIR = os.getenv("OUTPUT_DIR")

In [ ]:
# load the labels data with folds
import numpy as np

from utils.load_data import load_data
from data_models.Label import Label

label_path = os.path.join(DATA_DIR, "labels/labels.csv")
fold_path = os.path.join(DATA_DIR, "folds.json")

df = load_data(label_path=label_path, fold_path=fold_path)
df = df.set_index("specimen_id")
labels_onehot = df[Label._member_names_].to_dict(orient="split", index=True)
labels_onehot = {
    k: np.array(labels_onehot["data"][i])
    for i, k in enumerate(labels_onehot["index"])
}
labels_dict = {row.name: int(row["label"]) for _, row in df.iterrows()}

In [ ]:
# get the absolute path for each slide's set of tile embeddings
# tile_embed_dir = "/opt/gpudata/skin-cancer/outputs/prism/tile_embeddings"
# fnames = os.listdir(tile_embed_dir)
# tile_embed_paths = [
#     os.path.join(tile_embed_dir, fname)
#     for fname in fnames
#     if fname.endswith(".pkl") and fname[:6] in set(df.index)
# ]

In [ ]:
# from utils.tile_embed_postproc import postproc_pkl

# postproc_pkl(tile_embed_dir, tile_embed_dir + "_sorted", 256)

In [ ]:
tile_embed_dir = (
    "/opt/gpudata/skin-cancer/outputs/prism/tile_embeddings_sorted"
)
fnames = os.listdir(tile_embed_dir)
tile_embed_paths = [
    os.path.join(tile_embed_dir, fname)
    for fname in fnames
    if fname.endswith(".pkl") and fname[:6] in set(df.index)
]

In [ ]:
# import pickle

# with open(tile_embed_paths[0], "rb") as f:
#     data = pickle.load(f)

# data

In [ ]:
# get list of specimens within each fold
specimens_by_fold = df.groupby("fold").groups
specimens_by_fold = [list(specs) for specs in specimens_by_fold.values()]

In [ ]:
# map specimens to slides
slides_by_specimen = {spec: [] for spec in list(df.index)}
for slide in tile_embed_paths:
    slide_name = os.path.basename(slide)[:-4]
    spec = slide_name[:6]
    if slides_by_specimen.get(spec) is not None:
        slides_by_specimen[spec].append(slide)

In [ ]:
from data_models.Label import Label

class_freqs = {
    label: df[label].value_counts(normalize=True).iloc[1]
    for label in Label._member_names_
}

In [ ]:
import torch

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device {device}")

In [ ]:
from typing import Dict, List, Tuple

from torch.utils.data import DataLoader

from data_models.datasets import SlideEncodingDataset, collate_tile_embeds
from utils.split import train_val_split_slides, train_val_split_labels


def get_loaders(
    val_fold: int,
    specimens_by_fold: List[List[str]],
    slides_by_specimen: Dict[str, List[str]],
    labels_by_specimen: Dict[str, int],
) -> Tuple[DataLoader, DataLoader]:
    train, val = train_val_split_slides(
        val_fold=val_fold,
        specimens_by_fold=specimens_by_fold,
        slides_by_specimen=slides_by_specimen,
    )
    train_labels, val_labels = train_val_split_labels(
        val_fold=val_fold,
        labels_by_specimen=labels_by_specimen,
        specimens_by_fold=specimens_by_fold,
    )

    train_loader = DataLoader(
        SlideEncodingDataset(train, train_labels),
        batch_size=1,
        shuffle=True,
        collate_fn=collate_tile_embeds,
    )
    val_loader = DataLoader(
        SlideEncodingDataset(val, val_labels),
        batch_size=1,
        shuffle=False,
        collate_fn=collate_tile_embeds,
    )

    return train_loader, val_loader

In [ ]:
import copy
from operator import itemgetter

from torch import nn
from torch.optim import AdamW

from models.agg import MILClassifier
from models.utils.train import train_epoch, val_epoch
from utils.eval import Evaluator, get_spec_level_probs


EPOCHS = 30
BATCH_SIZE = 16
NUM_LABELS = 4
PATIENCE = 10

gated = False
heads = 1
model_name = "chkpts/prism-abmil-1_head-fold-{i}.pt"
exp_name = "prism/abmil/ungated/prism-3_hidden"

evaluator = Evaluator(Label)

for i in range(len(specimens_by_fold)):
    print(f"--------------------FOLD    {i + 1}--------------------")
    # get dataloaders and embed dims for loaded embeddings
    train_loader, val_loader = get_loaders(
        i, specimens_by_fold, slides_by_specimen, labels_dict
    )
    for sample in train_loader:
        embed_dim = sample["tile_embeds"].shape[-1]
        break

    model = MILClassifier(embed_dim, NUM_LABELS, heads, gated).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optim = AdamW(model.parameters(), lr=1e-5)

    best_loss = float("inf")
    best_model_weights = None
    best_model_data = {"ids": None, "labels": None, "probs": None}
    patience = PATIENCE

    for epoch in range(EPOCHS):
        train_loss = train_epoch(
            model, train_loader, optim, loss_fn, BATCH_SIZE
        )
        val_loss, labels, probs, ids = val_epoch(model, val_loader, loss_fn)

        if val_loss < best_loss:
            best_loss = val_loss
            best_model_weights = copy.deepcopy(model.state_dict())
            best_model_data["ids"] = ids
            best_model_data["labels"] = labels
            best_model_data["probs"] = probs
            patience = PATIENCE
        else:
            patience -= 1
            if patience == 0:
                break

        if (epoch + 1) % 2 == 0:
            spaces = " " * (4 - len(str(epoch + 1)))
            print(
                f"--------------------EPOCH{spaces}{epoch + 1}--------------------"
            )
            print(f"train loss: {train_loss:0.6f}")
            print(f"val loss:   {val_loss:0.6f}")
            print()

    # save the best model
    torch.save(
        best_model_weights,
        os.path.join(OUTPUT_DIR, model_name.format(i=i)),
    )

    # extract relevant data from val results for best model
    ids, probs = get_spec_level_probs(
        best_model_data["ids"], best_model_data["probs"]
    )
    labels_onehot_val = np.array(itemgetter(*ids)(labels_onehot))

    evaluator.fold(probs, labels_onehot_val, i, len(specimens_by_fold))

evaluator.finalize(class_freqs)
evaluator.save_figs(exp_name)